# SMT

## Import and utils

In [1]:
from z3 import *

## Read values from the file

In [2]:
# Read the file and initialize the variables
with open("../data/inst01.dat", "r") as file:
    lines = file.readlines()

In [3]:
# Parse the values from the file
m = int(lines[0].strip())  # Number of couriers
n = int(lines[1].strip())  # Number of items
l_values = list(map(int, lines[2].split()))  # Maximum load for each courier
l = [Int(f"l_{i}") for i in range(1, m + 1)]
for i in range(m):
    l[i] = l_values[i]

s_values = list(map(int, lines[3].split()))  # Size of each item
s = [Int(f"s_{i}") for i in range(1, n + 1)]
for i in range(n):
    s[i] = s_values[i]

D_values = [list(map(int, line.split())) for line in lines[4:]]  # Distance matrix
D = [[Int(f"D_{i}_{j}") for j in range(n + 1)] for i in range(n + 1)]
for i in range(n + 1):
    for j in range(n + 1):
        D[i][j] = D_values[i][j]

## Decision variables

In [4]:
# Array theory
routes = Array('routes', IntSort(), ArraySort(IntSort(), IntSort()))  # Routes for each courier
D = Array('D', IntSort(), ArraySort(IntSort(), IntSort()))  # Routes for each courier

# Set the matrix od distances D toi the default values
for i in range(n+1):
    a = Array('a', IntSort(), IntSort())
    for j in range(n+1):
        Store(a,j,D_values[i][j])
    Store(D,i,a)

# Accessing elements of routes
for i in range(m):
    for j in range(n+1):
        route = routes[i][j]
        # Use route in your constraints or expressions

In [10]:
# CREATE SOLVER INSTANCE
#solver = Solver()
solver = Optimize()

In [11]:
# DOMAIN CONSTRAINTS
for i in range(m):
    for t in range(n+1):
        solver.add(And(routes[i][t] >= 1, routes[i][t] <= n + 1))

## Constraints

In [22]:
# TODO: FIX CONSTRAINTS
# All the values from 1 to n need to appear exactly once in the routes matrix
for i in range(n):
    solver.add([If(routes[i][j] == i, 1, 0) for j in range(n+1)].count(1) == 1)

# The courier must finish at the origin point
for i in range(m):
    solver.add(routes[i][n+1] == n+1)

for i in range(m):
    # Constraint to enforce the total size of items assigned to each courier
    solver.add(Sum([If(routes[i][j] != n+1, s[routes[i][j]-1], 0) for j in range(n+1)]) <= l[i])

    # Each couriers size needs to be at least the minimum value of the items sizes (forces all couriers to have at least one item)
    solver.add(Sum([If(routes[i][j] != n+1, s[routes[i][j]-1], 0) for j in range(n+1)]) >= min(s))

# IMPLICIT CONSTRAINT
# Constraint to force exactly (m * (n + 1)) - n n+1s
solver.add([If(routes[i][j] == n + 1, 1, 0) for i in range(m) for j in range(n+1)].count((m * (n + 1)) - n) == 1)

TypeError: list indices must be integers or slices, not ArithRef

In [12]:
# OBJECTIVE FUNCTION
# Application of the same objective function above to all couriers
dist_courier = [Sum([D[routes[i][j]][routes[i][j + 1]] for j in range(n)]) + D[n + 1][routes[i][0]] for i in range(m)]

def z3_max(vector):
    maximum = vector[0]
    for value in vector[1:]:
        maximum = If(value > maximum, value, maximum)
    return maximum
maximum = z3_max(dist_courier)


In [13]:
solver.minimize(maximum)
if solver.check() == sat:
    model = solver.model()
    length = model.evaluate(maximum).as_string()
    
    routes_sol = []
    for i in range(m):
        routes_sol.append([model.evaluate(routes[i][j]).as_long() for j in range(n+1)])
    print(routes_sol)
    print(length)
else:
    print("no sat")
#TODO: Create function that prints the results


[[7, 7, 3, 7, 3, 4, 6], [7, 4, 6, 5, 7, 6, 5]]
-84499
